## Re-ranking por relevancia marginal máxima (MMR)

MMR es un método de re-ranking que combina la relevancia y la diversidad de los documentos recuperados. El objetivo es maximizar la relevancia de los documentos devueltos y minimizar la redundancia entre ellos.

Esto puede ser útil para incrementar la habilidad de los modelos de lenguage para generar respuestas con mayor cobertura y profundidad.

Su algoritmo es el siguiente:

1. Calcular los `embeddings` para cada documento y para la consulta.
2. Seleccionar el documento más relevante para la consulta.
3. Para cada documento restante, calcular el promedio de similitud de los documentos ya seleccionados.
4. Seleccionar el documento que es, en promedio, menos similar a los documentos ya seleccionados.
5. Repitir los pasos 3 y 4 hasta que se hayan seleccionado `k` documentos. Es decir, una lista ordenada que parte del documento que más contribuye a la diversidad general hasta el documento que contribuye menos.

En Langchain, el algoritmo de MMR es utilizado después de que el `retriever` ha recuperado los documentos más relevantes para la consulta. Por lo tanto, nos aseguramos que estamos seleccionando documentos diversos de un conjunto de documentos que ya son relevantes para la consulta.

![Re-ranking with MMR](./diagrams/slide_diagrama_04_V2.png)

## Librerías

In [1]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

from src.langchain_docs_loader import load_langchain_docs_splitted

load_dotenv()

False

## Carga de datos

In [3]:
docs = load_langchain_docs_splitted()

## Creación de retriever

Normalmente creamos nuestro retriever de la siguiente manera:

In [6]:
similarity_retriever = Chroma.from_documents(
    documents=docs[:100],
    embedding=OpenAIEmbeddings(),
).as_retriever(k=4)

Sin embargo, podrás notar que de hacerlo, el tipo de búsqueda que se realiza es por similitud de vectores. En este caso, queremos realizar una búsqueda por similitud de vectores, pero con un re-ranking por relevancia marginal máxima (MMR).

In [7]:
similarity_retriever.search_type

'similarity'

Entonces, para crear un retriever con re-ranking por MMR, debemos hacer lo siguiente:

In [8]:
mmr_retriever = Chroma.from_documents(
    documents=docs[:100],
    embedding=OpenAIEmbeddings(),
).as_retriever(
    search_type="mmr",
    k=4,  # number of documents to retrieve after mmr
    fetch_k=20,  # number of documents to fetch in the first step
    # Lambda mult is a number between 0 and 1 that determines the degree
    # of diversity among the results with 0 corresponding to maximum diversity
    # and 1 to minimum diversity.
    lambda_mult=0.5,
)

Ahora nuestro retriever está listo para ser utilizado con re-ranking por MMR.

In [9]:
mmr_retriever.search_type

'mmr'

## Uso del retriever

In [10]:
similarity_retriever.get_relevant_documents(
    "How to integrate LCEL into my Retrieval augmented generation system with a keyword search retriever?"
)

[Document(page_content='## RAG Search Example\u200b\n\nFor our next example, we want to run a retrieval-augmented generation\nchain to add some context when responding to questions.\n\n```python\n# Requires:\n# pip install langchain docarray tiktoken\n\nfrom langchain_community.vectorstores import DocArrayInMemorySearch\nfrom langchain_core.output_parsers import StrOutputParser\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain_core.runnables import RunnableParallel, RunnablePassthrough\nfrom langchain_openai.chat_models import ChatOpenAI\nfrom langchain_openai.embeddings import OpenAIEmbeddings\n\nvectorstore = DocArrayInMemorySearch.from_texts(\n    ["harrison worked at kensho", "bears like to eat honey"],\n    embedding=OpenAIEmbeddings(),\n)\nretriever = vectorstore.as_retriever()\n\ntemplate = """Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n"""\nprompt = ChatPromptTemplate.from_template(template)\nmodel = ChatOpen

In [11]:
mmr_retriever.get_relevant_documents(
    "How to integrate LCEL into my Retrieval augmented generation system with a keyword search retriever?"
)

[Document(page_content='## RAG Search Example\u200b\n\nFor our next example, we want to run a retrieval-augmented generation\nchain to add some context when responding to questions.\n\n```python\n# Requires:\n# pip install langchain docarray tiktoken\n\nfrom langchain_community.vectorstores import DocArrayInMemorySearch\nfrom langchain_core.output_parsers import StrOutputParser\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain_core.runnables import RunnableParallel, RunnablePassthrough\nfrom langchain_openai.chat_models import ChatOpenAI\nfrom langchain_openai.embeddings import OpenAIEmbeddings\n\nvectorstore = DocArrayInMemorySearch.from_texts(\n    ["harrison worked at kensho", "bears like to eat honey"],\n    embedding=OpenAIEmbeddings(),\n)\nretriever = vectorstore.as_retriever()\n\ntemplate = """Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n"""\nprompt = ChatPromptTemplate.from_template(template)\nmodel = ChatOpen